In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from pathlib import Path
import gc

project_dir = Path("notebook.ipynb").resolve().parents[0]

In [2]:
for file in os.listdir(f'{project_dir}/melted_data/'):
    print(file)

taxa_aprovacao_ensino_medio_value.csv
populacao_total_value.csv
total_concluintes_ensino_superior_value.csv
energia_total_value.csv
taxa_analfabetismo_maiores_quinze_anos_value.csv
total_concluintes_ensino_medio_value.csv
taxa_abandono_ensino_medio_value.csv
energia_total_consumidores_value.csv
taxa_reprovacao_ensino_medio_value.csv
taxa_abandono_ensino_fundamental_value.csv
total_concluintes_ensino_fundamental_value.csv
arrecadao_estadual_total_value.csv
taxa_reprovacao_ensino_fundamental_value.csv
estimativa_total_populacao_value.csv
pib_total_value.csv
arrecadacao_tributos_federais_value.csv
total_acesso_internet_value.csv
taxa_aprovacao_ensino_fundamental_value.csv


In [127]:
total=0
initial_data = pd.read_csv('data/pib_per_capita_2010_2018.csv', sep=';',decimal=',', skiprows=5, encoding='latin-1')
initial_data = initial_data.drop(columns=['latitude','longitude'])

for column in initial_data.columns[2:]:
    column_name = re.sub(r'\D*','',column)
    initial_data = initial_data.rename(columns={column:f'{column_name}'})

var_name = re.findall(r'^[A-Za-z_]*','pib_per_capita_2010_2018.csv')[0][:-1]
melted_data = initial_data.melt(id_vars=['Município','ibge'], var_name=f'ano', value_name=f'{var_name}_value')
melted_data = melted_data.astype({'ibge':'int','ano':'int'})
print(f'Tamanho inicial: {melted_data.shape}')
for file in os.listdir(f'{project_dir}/data/'):
    if file == ('pib_per_capita_2010_2018.csv'):
        continue
    elif file == 'TarifaMediaFornecimento.csv':
        continue
    else:
        data= pd.read_csv(f'{project_dir}/data/{file}',sep=';',decimal=',', skiprows=5, encoding='latin-1')
        data= data.drop(columns=['latitude','longitude'])
        var_name = re.findall(r'^[A-Za-z_]*',file)[0][:-1]

        for column in data.columns[2:]:
            column_name = re.sub(r'\D*','',column)
            data = data.rename(columns={column:f'{column_name}'})
        
        data_melted = data.melt(id_vars=['Município','ibge'], var_name=f'ano', value_name=f'{var_name}_value')
        #data_melted = data_melted[f'{var_name}_value']
        data_melted.to_csv(f'{project_dir}/melted_data/{var_name}_value.csv', sep=';', decimal=',', encoding='latin-1', index=False)
    total+= 1

for file in os.listdir(f'{project_dir}/melted_data/'):
    df= pd.read_csv(f'{project_dir}/melted_data/{file}', decimal=',',sep=';', encoding='latin-1')
    df = df.astype({'ibge':'int','ano':'int'})
    melted_data = melted_data.merge(df, on=['ibge','ano'], suffixes=('','_D'), how='outer')
    melted_data = melted_data.drop(melted_data.filter(regex='_D$').columns.tolist(), axis=1)
    melted_data.to_csv(f'{project_dir}/preliminar_dataset.csv', sep=';',decimal=',', encoding='latin-1')

final_dataset = melted_data
final_dataset.to_csv(f'{project_dir}/final_dataset.csv', sep=';',decimal=',', encoding='latin-1')
print(f'Tamanho final: {final_dataset.shape}')

Tamanho inicial: (4473, 4)
Tamanho final: (4473, 22)


In [139]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4473 entries, 0 to 4472
Data columns (total 22 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Município                                     4473 non-null   object 
 1   ibge                                          4473 non-null   int64  
 2   ano                                           4473 non-null   int64  
 3   pib_per_capita_value                          4473 non-null   object 
 4   taxa_aprovacao_ensino_medio_value             4464 non-null   object 
 5   populacao_total_value                         496 non-null    float64
 6   total_concluintes_ensino_superior_value       800 non-null    object 
 7   energia_total_value                           4149 non-null   object 
 8   taxa_analfabetismo_maiores_quinze_anos_value  496 non-null    float64
 9   total_concluintes_ensino_medio_value          2485 non-null   o